<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Stock_Analysis/Nifty100_Upside_Analysis_Forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance
!pip install talib-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np

mypf = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Stock_Analysis/data/Nifty100Scrips.csv')

mypf = mypf[mypf['InScope'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol']

#stock_ids

stock_ids = stock_n100

In [3]:
mypf = mypf[mypf.Forecast.notnull()]
stock_ids = mypf[mypf.Forecast.notnull()]['Symbol'].unique()

In [4]:
stock_ids.sort()
stock_ids

array(['ASIANPAINT', 'AXISBANK', 'BAJFINANCE', 'BERGEPAINT', 'BHARTIARTL',
       'BRITANNIA', 'CANBK', 'CIPLA', 'COLPAL', 'DABUR', 'DEEPAKNTR',
       'DIVISLAB', 'DMART', 'DRREDDY', 'EICHERMOT', 'FEDERALBNK',
       'GODREJCP', 'GODREJIND', 'HAVELLS', 'HCLTECH', 'HDFC', 'HDFCBANK',
       'HDFCLIFE', 'HEROMOTOCO', 'HINDUNILVR', 'IBREALEST', 'ICICIBANK',
       'ICICIPRULI', 'INDIGO', 'INDUSINDBK', 'INFY', 'IRCTC', 'ITC',
       'JUBLFOOD', 'KOTAKBANK', 'LT', 'LTI', 'M&M', 'MARUTI',
       'MCDOWELL-N', 'MINDTREE', 'MOTILALOFS', 'MUTHOOTFIN', 'NATIONALUM',
       'NAUKRI', 'PIDILITIND', 'RELIANCE', 'SAIL', 'SBILIFE', 'SBIN',
       'SUNPHARMA', 'TATACONSUM', 'TATAMOTORS', 'TATAPOWER', 'TATASTEEL',
       'TCS', 'TECHM', 'TITAN', 'ULTRACEMCO', 'WIPRO', 'ZEEL'],
      dtype=object)

In [5]:
mypf['InPortfolio'].value_counts()

0    44
1    17
Name: InPortfolio, dtype: int64

In [6]:
mypf[(mypf['Symbol'] + '.NS') == (stock_ids[1] + '.NS')]['Forecast'].values[0]

1044.0

In [7]:
# import necessary libraries 

%matplotlib inline
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
import talib 
yfin.pdr_override()

In [8]:
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
    #end_date = '2020-01-01'
    end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
    short_window = 20 
    mid_window = 50
    long_window = 200
    moving_avg = 'EMA'


    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-'))) 
    stock_df = web.DataReader(stock_symbol,  data_source='yahoo', start = start, end = end)['Close']
    stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe 
    stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows 

    # column names for long and short moving average columns
    short_window_col = str(short_window) + '_' + moving_avg
    mid_window_col = str(mid_window) + '_' + moving_avg
    long_window_col = str(long_window) + '_' + moving_avg  

    # Create short exponential moving average column
    stock_df[short_window_col] = round(stock_df['Close Price'].ewm(span = short_window, adjust = False).mean(),0)

    stock_df[mid_window_col] = round(stock_df['Close Price'].ewm(span = mid_window, adjust = False).mean(),0)     

    stock_df['RSI'] = talib.RSI(stock_df['Close Price'], timeperiod=14)

    # Create a long exponential moving average column
    stock_df[long_window_col] = round(stock_df['Close Price'].ewm(span = long_window, adjust = False).mean(),0)
    stock_df['Stock'] = stock_symbol
    stock_df['Dev%_200'] = round((stock_df['Close Price'] - stock_df['200_EMA'])*100/stock_df['200_EMA'],2)
    stock_df['Max'] = round(max(stock_df['Close Price']),0)
    stock_df['Min'] = round(min(stock_df['Close Price']),0)    
    #stock_df['Dev%_Max'] = round((stock_df['Close Price'] - stock_df['Max'])*100/stock_df['Max'],2)
    #stock_df['Dev%_Min'] = round((stock_df['Close Price'] - stock_df['Min'])*100/stock_df['Min'],2)
    stock_df['Premium%'] = round((stock_df['Close Price']-stock_df['Min'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df['Discount%'] = round((stock_df['Max']-stock_df['Close Price'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df['Close Price'] = round(stock_df['Close Price'],0) 
    
    
    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:
    stock_id = stock_id.upper() + '.NS'
    print(stock_id)
    tmp = stock_prec_dev(stock_id)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Forecast'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Forecast'].values[0]
    df_prec_dev = df_prec_dev.append(tmp)

ASIANPAINT.NS
[*********************100%***********************]  1 of 1 completed
AXISBANK.NS
[*********************100%***********************]  1 of 1 completed
BAJFINANCE.NS
[*********************100%***********************]  1 of 1 completed
BERGEPAINT.NS
[*********************100%***********************]  1 of 1 completed
BHARTIARTL.NS
[*********************100%***********************]  1 of 1 completed
BRITANNIA.NS
[*********************100%***********************]  1 of 1 completed
CANBK.NS
[*********************100%***********************]  1 of 1 completed
CIPLA.NS
[*********************100%***********************]  1 of 1 completed
COLPAL.NS
[*********************100%***********************]  1 of 1 completed
DABUR.NS
[*********************100%***********************]  1 of 1 completed
DEEPAKNTR.NS
[*********************100%***********************]  1 of 1 completed
DIVISLAB.NS
[*********************100%***********************]  1 of 1 completed
DMART.NS
[*******************

In [9]:
stock_prec_dev('^NSEI').tail(1)

[*********************100%***********************]  1 of 1 completed


,Close Price,20_EMA,50_EMA,RSI,200_EMA,Stock,Dev%_200,Max,Min,Premium%,Discount%
Date,,,,,,,,,,,
2022-11-17,18403.0,18041.0,17708.0,68.578075,17273.0,^NSEI,6.54,18410.0,15294.0,99.79,0.21


In [10]:
df_prec_dev['Upside%'] = round((df_prec_dev['Forecast']-df_prec_dev['Close Price'])*100/(df_prec_dev['Close Price']),2)
df_prec_dev = df_prec_dev.sort_values(by = 'Upside%', ascending=False)
stock_ids = df_prec_dev['Stock'].values
df_prec_dev.reset_index(drop=True, inplace=True)

In [11]:
# buy or accumulate (based on std dev from 200 EMA)
df_prec_dev.sort_values(by = 'Dev%_200', ascending=True).head(20)

,Date,Close Price,20_EMA,50_EMA,RSI,200_EMA,Stock,Dev%_200,Max,Min,Premium%,Discount%,Forecast,Upside%
6,2022-11-17,3304.0,3468.0,3566.0,36.099867,3878.0,DIVISLAB.NS,-14.79,4938.0,3271.0,1.99,98.01,4065.0,23.03
32,2022-11-17,397.0,393.0,398.0,54.626033,465.0,WIPRO.NS,-14.58,722.0,375.0,6.40,93.60,433.0,9.07
0,2022-11-17,409.0,421.0,434.0,32.210005,474.0,GODREJIND.NS,-13.69,644.0,406.0,1.30,98.70,638.0,55.99
1,2022-11-17,685.0,694.0,716.0,40.229991,789.0,MOTILALOFS.NS,-13.13,980.0,682.0,1.14,98.86,930.0,35.77
14,2022-11-17,81.0,79.0,78.0,56.683988,93.0,IBREALEST.NS,-12.90,181.0,60.0,17.36,82.64,95.0,17.28
10,2022-11-17,75.0,74.0,74.0,55.103769,85.0,NATIONALUM.NS,-11.35,130.0,67.0,13.25,86.75,91.0,21.33
2,2022-11-17,480.0,500.0,518.0,32.686940,540.0,ICICIPRULI.NS,-11.09,643.0,447.0,16.89,83.11,647.0,34.79
31,2022-11-17,1059.0,1053.0,1052.0,52.608012,1180.0,TECHM.NS,-10.25,1806.0,965.0,11.18,88.82,1156.0,9.16
7,2022-11-17,3951.0,3990.0,3992.0,48.885182,4355.0,NAUKRI.NS,-9.28,6545.0,3375.0,18.17,81.83,4851.0,22.78
13,2022-11-17,555.0,583.0,592.0,34.021019,603.0,JUBLFOOD.NS,-7.96,786.0,464.0,28.26,71.74,651.0,17.30


In [12]:
# sell or downsize (based on std dev from 200 EMA)
df_prec_dev.sort_values(by = 'Dev%_200', ascending=False).head(15)

,Date,Close Price,20_EMA,50_EMA,RSI,200_EMA,Stock,Dev%_200,Max,Min,Premium%,Discount%,Forecast,Upside%
52,2022-11-17,308.0,290.0,266.0,72.427852,235.0,CANBK.NS,30.98,314.0,175.0,95.54,4.46,310.0,0.65
27,2022-11-17,135.0,133.0,127.0,57.162373,110.0,FEDERALBNK.NS,22.55,138.0,81.0,94.39,5.61,149.0,10.37
49,2022-11-17,345.0,347.0,338.0,49.243119,297.0,ITC.NS,16.13,361.0,208.0,89.48,10.52,350.0,1.45
12,2022-11-17,601.0,586.0,563.0,62.618031,522.0,SBIN.NS,15.22,616.0,435.0,91.96,8.04,709.0,17.97
46,2022-11-17,848.0,819.0,792.0,68.989654,741.0,BHARTIARTL.NS,14.39,848.0,641.0,99.83,0.17,879.0,3.66
9,2022-11-17,860.0,854.0,820.0,55.354091,755.0,AXISBANK.NS,13.91,915.0,626.0,80.97,19.03,1044.0,21.40
26,2022-11-17,1270.0,1296.0,1276.0,43.668486,1125.0,M&M.NS,12.93,1354.0,691.0,87.41,12.59,1404.0,10.55
57,2022-11-17,4127.0,3968.0,3853.0,68.448519,3665.0,BRITANNIA.NS,12.62,4175.0,3094.0,95.60,4.40,3935.0,-4.65
15,2022-11-17,920.0,906.0,888.0,58.855416,818.0,ICICIBANK.NS,12.44,926.0,654.0,97.72,2.28,1075.0,16.85
42,2022-11-17,1014.0,1007.0,971.0,57.740124,902.0,SUNPHARMA.NS,12.36,1052.0,737.0,87.78,12.22,1056.0,4.14


In [13]:
# portfolio buy/sell
mypf['Stock'] = mypf['Symbol'] + '.NS'
df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]

,Date,Close Price,20_EMA,50_EMA,RSI,200_EMA,Stock,Dev%_200,Max,Min,Premium%,Discount%,Forecast,Upside%
0,2022-11-17,409.0,421.0,434.0,32.210005,474.0,GODREJIND.NS,-13.69,644.0,406.0,1.30,98.70,638.0,55.99
1,2022-11-17,685.0,694.0,716.0,40.229991,789.0,MOTILALOFS.NS,-13.13,980.0,682.0,1.14,98.86,930.0,35.77
2,2022-11-17,480.0,500.0,518.0,32.686940,540.0,ICICIPRULI.NS,-11.09,643.0,447.0,16.89,83.11,647.0,34.79
3,2022-11-17,534.0,535.0,540.0,48.317277,569.0,HDFCLIFE.NS,-6.19,711.0,504.0,14.40,85.60,683.0,27.90
4,2022-11-17,1256.0,1254.0,1251.0,51.009398,1205.0,SBILIFE.NS,4.24,1332.0,1025.0,75.29,24.71,1577.0,25.56
5,2022-11-17,1760.0,1776.0,1808.0,45.512894,1884.0,INDIGO.NS,-6.58,2332.0,1549.0,26.96,73.04,2186.0,24.20
6,2022-11-17,3304.0,3468.0,3566.0,36.099867,3878.0,DIVISLAB.NS,-14.79,4938.0,3271.0,1.99,98.01,4065.0,23.03
7,2022-11-17,3951.0,3990.0,3992.0,48.885182,4355.0,NAUKRI.NS,-9.28,6545.0,3375.0,18.17,81.83,4851.0,22.78
10,2022-11-17,75.0,74.0,74.0,55.103769,85.0,NATIONALUM.NS,-11.35,130.0,67.0,13.25,86.75,91.0,21.33
14,2022-11-17,81.0,79.0,78.0,56.683988,93.0,IBREALEST.NS,-12.90,181.0,60.0,17.36,82.64,95.0,17.28


In [14]:
df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]['Upside%']*100/df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]['Upside%'].sum()

0     16.518173
1     10.552868
2     10.263748
3      8.231060
4      7.540713
5      7.139485
6      6.794312
7      6.720557
10     6.292778
14     5.097947
21     3.372079
22     3.348478
24     3.165565
29     2.897097
43     1.191881
48     0.592990
51     0.280269
Name: Upside%, dtype: float64

In [15]:
# prospects buy
df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]

,Date,Close Price,20_EMA,50_EMA,RSI,200_EMA,Stock,Dev%_200,Max,Min,Premium%,Discount%,Forecast,Upside%
8,2022-11-17,1143.0,1151.0,1140.0,46.722369,1040.0,INDUSINDBK.NS,9.87,1264.0,778.0,75.02,24.98,1394.0,21.96
9,2022-11-17,860.0,854.0,820.0,55.354091,755.0,AXISBANK.NS,13.91,915.0,626.0,80.97,19.03,1044.0,21.40
11,2022-11-17,253.0,262.0,262.0,38.379833,264.0,ZEEL.NS,-4.07,371.0,209.0,27.31,72.69,302.0,19.37
12,2022-11-17,601.0,586.0,563.0,62.618031,522.0,SBIN.NS,15.22,616.0,435.0,91.96,8.04,709.0,17.97
13,2022-11-17,555.0,583.0,592.0,34.021019,603.0,JUBLFOOD.NS,-7.96,786.0,464.0,28.26,71.74,651.0,17.30
15,2022-11-17,920.0,906.0,888.0,58.855416,818.0,ICICIBANK.NS,12.44,926.0,654.0,97.72,2.28,1075.0,16.85
16,2022-11-17,425.0,421.0,422.0,52.618341,443.0,TATAMOTORS.NS,-4.16,530.0,372.0,33.26,66.74,493.0,16.00
17,2022-11-17,6929.0,7099.0,7125.0,40.085015,6933.0,BAJFINANCE.NS,-0.06,7853.0,5284.0,64.03,35.97,7978.0,15.14
18,2022-11-17,2465.0,2521.0,2550.0,38.268348,2451.0,HINDUNILVR.NS,0.57,2702.0,1944.0,68.73,31.27,2828.0,14.73
19,2022-11-17,842.0,837.0,850.0,50.391501,850.0,GODREJCP.NS,-1.00,968.0,665.0,58.25,41.75,958.0,13.78


In [16]:
# CCI buy
df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['CCIFlag'] == 1].Stock.values)]

,Date,Close Price,20_EMA,50_EMA,RSI,200_EMA,Stock,Dev%_200,Max,Min,Premium%,Discount%,Forecast,Upside%
6,2022-11-17,3304.0,3468.0,3566.0,36.099867,3878.0,DIVISLAB.NS,-14.79,4938.0,3271.0,1.99,98.01,4065.0,23.03
13,2022-11-17,555.0,583.0,592.0,34.021019,603.0,JUBLFOOD.NS,-7.96,786.0,464.0,28.26,71.74,651.0,17.30
18,2022-11-17,2465.0,2521.0,2550.0,38.268348,2451.0,HINDUNILVR.NS,0.57,2702.0,1944.0,68.73,31.27,2828.0,14.73
24,2022-11-17,1594.0,1538.0,1507.0,65.518764,1565.0,INFY.NS,1.88,1940.0,1365.0,39.90,60.10,1765.0,10.73
31,2022-11-17,1059.0,1053.0,1052.0,52.608012,1180.0,TECHM.NS,-10.25,1806.0,965.0,11.18,88.82,1156.0,9.16
32,2022-11-17,397.0,393.0,398.0,54.626033,465.0,WIPRO.NS,-14.58,722.0,375.0,6.40,93.60,433.0,9.07
34,2022-11-17,3085.0,3121.0,3191.0,42.020350,3172.0,ASIANPAINT.NS,-2.74,3576.0,2580.0,50.71,49.29,3361.0,8.95
35,2022-11-17,1224.0,1231.0,1254.0,46.179642,1262.0,HAVELLS.NS,-2.97,1418.0,1070.0,44.40,55.60,1327.0,8.42
37,2022-11-17,610.0,603.0,614.0,52.262848,652.0,BERGEPAINT.NS,-6.44,792.0,561.0,21.21,78.79,651.0,6.72
41,2022-11-17,2593.0,2670.0,2627.0,40.135241,2478.0,TITAN.NS,4.63,2770.0,1936.0,78.74,21.26,2715.0,4.70
